In [22]:
%matplotlib inline
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from mpl_toolkits.basemap import Basemap
import ipywidgets as widgets
import folium
from folium.plugins import HeatMap

In [23]:
def inline_map(m, width=650, height=500):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{}" '
                 'style="width: {}px; height: {}px; '
                 'border: none"></iframe>'.format(srcdoc, width, height))
    return embed

In [24]:
data = pd.read_csv('taxi_7_data.csv', sep=';', index_col='dt')

In [25]:
regions = pd.read_csv('taxi_7_regions.csv', sep=';', index_col=False)

In [26]:
regions.head()

,region,west,east,south,north,lon,lat
0,1075,-74.022246,-74.011135,40.697437,40.705825,-74.016691,40.701631
1,1076,-74.022246,-74.011135,40.705825,40.714213,-74.016691,40.710019
2,1077,-74.022246,-74.011135,40.714213,40.722601,-74.016691,40.718407
3,1125,-74.011135,-74.000023,40.697437,40.705825,-74.005579,40.701631
4,1126,-74.011135,-74.000023,40.705825,40.714213,-74.005579,40.710019


# Неделя 7. Оформление проекта в виде интерактивного демо.

Итоговые временные ряды были рассчитаны с помощью рекуррентной нейронной сети по методике, изложенной здесь - http://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

В этой демонстрации используются данные за май (реальные и прогнозные). Поскольку прогнозирование занимает довольно длительное время, то прогнозные данные были рассчитаны заранее и сохранены в датасет.

В соответствии с требованиями демо состоит из двух частей.

## 1. Карты с визуализацией реального и прогнозируемого спроса на такси в выбираемый пользователем момент времени

In [27]:
lat0, lon0 = -73.9778, 40.705825

def VisualizeDataForDT(day, hour):
    radars = folium.Map(location=[40, -122], zoom_start=5, tiles='OpenStreetMap')
    map_1 = folium.Map(location=[lat0, lon0], zoom_start=12)
    return inline_map(map_1)
    #inline_map(map_1)
    #html = inline(map_1, 900, 500)
    #for name, data in map.json_data.items():
    #    html = html.replace('d3.json, ' + repr(name), 'function(callback){callback(null, JSON.parse('+ repr(json.dumps(data).replace('"', '&quot;')) + '))}')
    #HTML(html)
    
   
    

In [28]:
def VisualizeForecastForDT(day, hour):
    print day, hour

In [29]:
WData = interactive(VisualizeDataForDT, day=widgets.IntSlider(min=1,max=31,step=1,value=1), hour=widgets.IntSlider(min=0,max=23,step=1,value=0))
WForecast = interactive(VisualizeForecastForDT, day=widgets.IntSlider(min=1,max=31,step=1,value=1), hour=widgets.IntSlider(min=0,max=23,step=1,value=0))

In [30]:
print 'Реальные данные'
display(WData)
print 'Прогнозные данные'
display(WForecast)

Реальные данные


Прогнозные данные
